In [1]:
from qiskit import IBMQ
from qiskit.opflow.primitive_ops import PauliOp
from qiskit.opflow.list_ops import SummedOp
from qiskit.quantum_info import Pauli
from qiskit.algorithms import VQE
import numpy as np

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') 

In [3]:
backend = provider.get_backend('ibmq_lima')
defaults = backend.defaults()

In [4]:
conexions = defaults.instruction_schedule_map.qubits_with_instruction('x') + [ indx for indx in defaults.instruction_schedule_map.qubits_with_instruction('cx') ]
conexions

[0, 1, 2, 3, 4, (0, 1), (1, 0), (1, 2), (1, 3), (2, 1), (3, 1), (3, 4), (4, 3)]

In [5]:
#Hamiltonian 
def HeisenbergHamiltonian( J=1, H=1, num_qubits=2, neighbours=[(0,1)] ):
    """
    Qiskit operator of the 3-D Heisemberg Hamiltonian of a lattice of spins.
    
    H = - J Σ_j ( X_j X_{j+1} + Y_j Y_{j+1} + Z_j Z_{j+1} ) - H Σ_j Z_j
    
    input:
        J          : Real. Coupling constant.
        H          : Real. External magnetic field.
        num_qubits : Integer. Number of qubits.
        neighbours : List of tuples. Coupling between the spins.
    output:
        Hamiltonian : SummedOp of Qiskit. Heisenberg Hamiltonian of the system.
    """
    num_op = num_qubits + 3*len(neighbours)
    Hamiltonian_op_x = []    
    Hamiltonian_op_z = []  
    Hamiltonian_coef = num_qubits*[-H] + num_op*[-J]
    
    for idx in range(num_qubits):
        op_x = np.zeros( num_qubits )
        op_z = np.zeros( num_qubits )
        op_z[idx] = 1
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )        
    
    for idx in neighbours:
        op_x = np.zeros( num_qubits )
        op_z = np.zeros( num_qubits )
        op_x[idx[0]] = 1
        op_x[idx[1]] = 1
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )
        op_z[idx[0]] = 1
        op_z[idx[1]] = 1
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )        
        op_x[idx[0]] = 0
        op_x[idx[1]] = 0
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )        
        
#     Hamiltonian = WeightedPauliOperator( 
#         [ [Hamiltonian_coef[j], Pauli( ( Hamiltonian_op_z[j], Hamiltonian_op_x[j] )) ] 
#          for j in range(num_op) ] )

    Hamiltonian = SummedOp( [ PauliOp(Pauli( ( Hamiltonian_op_z[j], Hamiltonian_op_x[j] )),Hamiltonian_coef[j]) for j in range(num_op) ] )

    return Hamiltonian


def RandomHamiltonian( num_qubits=2, num_paulis=4 ):
    
    idxs = np.random.randint(2, size=(2,num_qubits,num_paulis) )

    Hamiltonian = SummedOp( [ PauliOp(Pauli( ( idxs[0,:,j], idxs[1,:,j] )),1) for j in range(num_paulis) ] )
    
    return Hamiltonian

In [7]:
QubitOp = HeisenbergHamiltonian( num_qubits=4, neighbours=[(0,1),(1,2),(2,3),(3,0)] )
print( QubitOp )

SummedOp([
  -1 * IIIZ,
  -1 * IIZI,
  -1 * IZII,
  -1 * ZIII,
  -1 * IIXX,
  -1 * IIYY,
  -1 * IIZZ,
  -1 * IXXI,
  -1 * IYYI,
  -1 * IZZI,
  -1 * XXII,
  -1 * YYII,
  -1 * ZZII,
  -1 * XIIX,
  -1 * YIIY,
  -1 * ZIIZ
])


In [8]:
QubitOp = RandomHamiltonian(3,3)
print( QubitOp )

SummedOp([
  YXY,
  ZZX,
  XXZ
])


In [9]:
def Label2Chain(QubitOp):
    """
    Transform a string of Pauli matrices into a numpy array.
    'I' --> 0
    'X' --> 1
    'Y' --> 2
    'Z' --> 3
    
    input:
        QubitOp : SummedOp of Qiskit.
    output:
        ops     : narray of the Pauli operators.
                  ops.shape = ( number_of_operators, number_of_qubits )
        coef    : coefficient of each Pauli operator.
    """
    Dict = {'I' : 0,
            'X' : 1,
            'Y' : 2,
            'Z' : 3}
    ops = [[ Dict.get(idx2) for idx2 in idx.primitive.to_label()] for idx in QubitOp.oplist ]
    coef = [ idx.coeff for idx in QubitOp.oplist ]
    return np.array(ops), coef

In [10]:
paulis, coef = Label2Chain(QubitOp)
paulis

array([[2, 1, 2],
       [3, 3, 1],
       [1, 1, 3]])

In [11]:
import numpy as np
import networkx as nx
from itertools import permutations
import time

In [12]:
def PauliGraph(PS):#PS==Pauli Strings. AM=Admisible Measurements. WC==Well Connected Qubits. 
    #    If we want to group n Pauli arrays of size N, PS should be a matrix of n rows and N columns,
    # each row representing a Pauli string.
    n=np.size(PS[:,0])
    N=np.size(PS[0,:])
    G = nx.Graph()
    G.add_nodes_from(np.arange(n))
    for i in range(n):
        v_i=PS[i,:]
        for j in range(i+1,n):
            v_j=PS[j,:]
            qubits=np.arange(N)
            noncommonqubits=np.delete(qubits,np.argwhere(v_i==v_j))
            vi=v_i[noncommonqubits]
            vj=v_j[noncommonqubits]
            if (vi*vj!=0).any():
                G.add_edges_from([(i,j)])
    return G


def LDFC(PG):
    SV=sorted(PG.degree, key=lambda x: x[1], reverse=True)#Sorted Vertices by decreasing degree
    n=PG.number_of_nodes()
    aux=list(np.arange(n))
    Color=n*np.ones(n)
    for i in range(n):
        IV=list(list(PG.neighbors(SV[i][0])))#Vertices that are Incompatible with vertex SV[i][0]
        IC=Color[IV]#Colors that are assigned to vertices that are incompatible with vertex SV[i]
        AC=[ elem for elem in aux if elem not in IC]#Available colors for vertex SV[i]
        Color[SV[i][0]]=min(AC)
    MC=int(max(Color))
    Groups=[]
    for i in range(MC+1):
        Groups.append(list(np.argwhere(Color==i)))
    return Color, Groups #Color is an array whose i entry has the color assigned to the i Pauli String.
    #Groups is a list of lists, where the i list comprenhends the arrays assigned to the color i.
        
def TPBgrouping(PS): #PS==Pauli Strings. AM=Admisible Measurements. WC==Well Connected Qubits. 
    #    If we want to group n Pauli arrays of size N, PS should be a matrix of n rows and N columns,
    # each row representing a Pauli string.
    PG=PauliGraph(PS)
    Color, Groups=LDFC(PG)
    N=np.size(PS[0,:])
    Measurements=[]#The list of measurements. Each element will be the total measurement for a certain group. That measurement 
    #will be encoded as an N-array of {0,1,3,4}. 0 will appear in the position k if in the qubit k we can measure with any 
    # basis (will only happen if the k factor of every element of the group is I), 0 will appear in the position k if in the qubit k
    #we can measure with TPBX,...
    for i in range(len(Groups)):
        Mi=[]
        for k in range(N):
            Mi.append(max(PS[Groups[i],k]))
        Measurements.append(Mi)
    return Color, Groups, Measurements


In [13]:
Comp=[]
Comp.append([0,1])
Comp.append([0,2])
Comp.append([0,3])
Comp.append([[0,0],[1,1],[2,2],[3,3]])
length=[]
length.append(1)
length.append(1)
length.append(1)
length.append(2)

In [14]:
def MeasurementAssignment(Vi,Vj,Mi,AM,WC):#This program is the Algorithm 2 of https://arxiv.org/abs/1909.09119. Syntax can
    #be looked in 'grouping(PS,AM,WC)'
    
    # Let's first check for compatibility of Vj with the current assigment of Mi.
    # Mi is a list of local measurement. Each local measurement is encoded as list of two elements. The first one 
    # are the qubits where the local measurement acts and the second is the type of local measurement. For example,
    # if Mi contains {4,(1,2)} it would mean that Mi has the Bell measurement (nº4) as the local measurement acting on 
    # the qubits (1,2).
    N=np.size(Vi)
    U=list(np.arange(N))
    S=0
    for LM in Mi:
        if list(Vj[LM[1]]) not in Comp[LM[0]]:
            return Mi, S
        else:
            for s in LM[1]:### SEGURO QUE HAY UNA FORMA MÁS RÁPIDA DE ELIMINAR VARIOS VALORES A LA VEZ DE LA LISTA
                U.remove(LM[1][s])
    commonfactors=np.argwhere(Vi==Vj)
    for k in commonfactors:
        U.remove(k)
    PMi=Mi #I create a potential Mi.
    while len(U)!=0:   
        for Eps in AM:
            if len(U)>=length[Eps]:
                perm=list(permutations(U,length[Eps])) #length of each local measurement will be manually programmed
                perm=list({*map(tuple, map(sorted, perm))}) #This is a code for eliminating the permutations that
                #are equal up to order for perm. This would reduce the iterations (I believe) without affecting the algorithm,
                #because the WC array will contain all possible permutations, even those that are equal with disitinct order.
                #and if the qubits (l,k) of Vi and Vj are compatible with a certain measurement, the qubits (k,l) of Vi and 
                #Vj will be compatible with other measurement. I should explain this better. 
                for per in perm:
                    #if per in WC: 
                    #This raises an error, so here I have to check the code. 
                    if (list(Vi[[per]]) in Comp[Eps]) and (list(Vj[[per]]) in Comp[Eps]):
                        PMi.append([Eps,list(per)])
                        for s in range(np.size(per)):
                            U.remove(per[s])
                        break
                else:
                    continue
                break
        else:
            continue
            return Mi, S
    S=1
    return PMi, S            

def grouping(PS, AM, WC): #PS==Pauli Strings. AM=Admisible Measurements. WC==Well Connected Qubits. 
    #    If we want to group n Pauli arrays of size N, PS should be a matrix of n rows and N columns,
    # each row representing a Pauli string. 
    #    AM should be a vector containing the admisible measurements in the order of prefered assignenment. 
    #    WC should be a vector containing the pairs of qubits with good connectivity.
    PG=PauliGraph(PS)
    SV=sorted(PG.degree, key=lambda x: x[1], reverse=True)#Sorted Vertices by decreasing degree
    n=np.size(PS[:,0])
    N=np.size(PS[0,:])
    AS=[]#list of strings with assigned measurement
    Groups=[]#list of groups
    Measurements=[]#list of total measurements Mi
    for k in range(n):
        print('k',k)
        i=SV[k][0]
        if i not in AS:
            print('i',i)
            Mi=[]#Mi will be the total measurement. It will be a list of local measurements. Each local measurement
            #will appear as a list of two elements. The first will correspond with the local measurement and the second
            # to the qubits. For example, if Mi contains {4,(1,2)} it would mean that Mi has the Bell measurement (nº4)
            #as the local measurement acting on the qubits (1,2)
            GroupMi=[i]
            AS.append(i)
            for l in range(n):
                print('l',l)
                j=SV[l][0]
                if j not in AS:
                    Mi, S=MeasurementAssignment(PS[i,:],PS[j,:],Mi,AM,WC)#S is the success variable. If Mi is compatible with
                    #Vj S=1 otherwise S=0
                    if S==1:
                        AS.append(j)
                        GroupMi.append(j)
            QWM=list(np.arange(N))#Qubits Without a Measurement assigned by Mi. There, all factors 
            # of the group will be equal or the identity, so we will have to use a TPB measurement.
            for LM in Mi:
                for s in LM[1]:
                    QWM.remove(s)
            for q in QWM:
                TPBq=max(PS[GroupMi,q])
                Mi.append([TPBq-1,q])
            Groups.append(GroupMi)
            Measurements.append(Mi)
            
    return Groups, Measurements

In [15]:
Color, Groups, Measurements = TPBgrouping(PS)

NameError: name 'PS' is not defined

In [ ]:
Color

In [ ]:
Groups

In [ ]:
Measurements

In [16]:
PS=np.array([[1,1,3],[2,2,3],[3,3,3],[1,2,3]])
WC=list(np.arange(np.size(PS[0,:])))#One should include all qubits in the WC array, othewise the if loop which checks if 
#the measurement is done in compatible qubits would reject all one qubit measurements.
WC2=permutations(WC,2)#let's suppose that all qubits are pairwise connected
WC.extend(WC2)
Groups, Measurements = grouping(PS,[3],WC)
Groups, Measurements

k 0
i 0
l 0
l 1
l 2
l 3
k 1
k 2
k 3
i 3
l 0
l 1
l 2
l 3


C:\Users\Luciano\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:34: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


([[0, 1, 2], [3]], [[[3, [0, 1]], [2, 2]], [[0, 0], [1, 1], [2, 2]]])

In [18]:
QubitOp = HeisenbergHamiltonian( num_qubits=4, neighbours=[(0,1),(1,2),(2,3),(3,0)] )
paulis, coef = Label2Chain(QubitOp)
print( paulis )

[[0 0 0 3]
 [0 0 3 0]
 [0 3 0 0]
 [3 0 0 0]
 [0 0 1 1]
 [0 0 2 2]
 [0 0 3 3]
 [0 1 1 0]
 [0 2 2 0]
 [0 3 3 0]
 [1 1 0 0]
 [2 2 0 0]
 [3 3 0 0]
 [1 0 0 1]
 [2 0 0 2]
 [3 0 0 3]]


In [19]:
WC = list(np.arange(4))  #One should include all qubits in the WC array, othewise the if loop which checks if 
#the measurement is done in compatible qubits would reject all one qubit measurements.
WC2 = permutations(WC,2) #let's suppose that all qubits are pairwise connected
WC.extend(WC2)
WC

[0,
 1,
 2,
 3,
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2)]

In [20]:
Groups, Measurements = grouping(paulis,[3],WC)

k 0
i 4
l 0
l 1
l 2
l 3
l 4


C:\Users\Luciano\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:34: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


IndexError: list index out of range

In [167]:
Groups

[[0, 1, 2], [3]]

In [133]:
Measurements

[[[3, [0, 1]], [2, 2]], [[0, 0], [1, 1], [2, 2]]]